# Семинар 8

## 1. Динамическое программирование (продолжение)

### 1.1. Рюкзак 0-1 (Рекурсия)

In [20]:
def knapsack_recursive(W: int, arr: list, n: int) -> int:
    if n == 0 or W == 0:
        return 0
    
    if arr[n-1][1] > W:
        return knapsack_recursive(W, arr, n-1)
    else:
        return max(
            arr[n-1][0] + knapsack_recursive(W - arr[n-1][1], arr, n-1),
            knapsack_recursive(W, arr, n-1)
        )

arr = [(7, 1), (3, 3), (1, 2), (2, 6), (2, 5)] # выдает правильный ответ
x = 10
knapsack_recursive(x, arr, len(arr))

12

### 1.2. Рюкзак 0-1 (Рекурсия с мемоизацией)

In [23]:
def knapsack_memo(W: int, arr: list, n: int, t: list) -> int:
    if n == 0 or W == 0:
        return 0
    
    if t[n][W] != -1:
        return t[n][W]
    
    if arr[n-1][1] <= W:
        t[n][W] = max(
            arr[n-1][0] + knapsack_memo(
                W-arr[n-1][1], arr, n-1, t),
            knapsack_memo(W, arr, n-1, t)
        )
        return t[n][W]
    elif arr[n-1][1] > W:
        t[n][W] = knapsack_memo(W, arr, n-1, t)
        return t[n][W]

arr = [(7, 1), (3, 3), (1, 2), (2, 6), (2, 5)]
x = 10
n = 5
t = [[-1 for i in range(x + 1)] for j in range(n + 1)]
knapsack_memo(x, arr, len(arr), t)

12

### 1.3. Рюкзак 0-1, (Динамически)

In [26]:
def knapsack_dynamic(W, arr, n) -> int:
    K = [[0 for x in range(W+1)] for x in range(n + 1)]
    for i in range(n + 1):
        for w in range(W + 1):
            if i == 0 or w == 0:
                K[i][w] = 0
            elif arr[i-1][1] <= w:
                K[i][w] = max(
                    arr[i-1][0] + K[i-1][w-arr[i-1][1]],
                    K[i-1][w]
                )
            else:
                K[i][w] = K[i-1][w]
    return K[n][W]

arr = [(7, 1), (3, 3), (1, 2), (2, 6), (2, 5)]
x = 10
knapsack_dynamic(x, arr, len(arr))

12

### 2. Наибольшая общая подпоследовательность

In [42]:
def lcs(s1: str, s2: str) -> int:
    n, m = len(s1), len(s2)
    dp = [[0 for _ in range(len(s2)+1)] for _ in range(len(s1)+1)] # можно лучше и меньше памяти
    for ind1 in range(1, n+1):
        for ind2 in range(1, m+1):
            if s1[ind1-1] == s2[ind2-1]:
                dp[ind1][ind2] = 1 + dp[ind1][ind2-1]
            else:
                dp[ind1][ind2] = max(dp[ind1][ind2-1], dp[ind1-1][ind2])
    return dp[n][m]

s1 = 'abcdkacd'
s2 = 'kgafe'
lcs(s1, s2)

2

### 3. Наибольшая возрастающая подпоследовательность

In [45]:
def lis(arr: list) -> int:
    n = len(arr)
    dp = [1] * n
    for i in range(1, n):
        for j in range(0, i):
            if arr[i] > arr[j] and dp[i] < dp[j] + 1:
                dp[i] = dp[j] + 1
    return max(dp)

arr = [0, 1, 3, 5, -1, -2, 8]
lis(arr)

5

## 2. Алгоритмы на строках

### 1. Расстояние Левенштейна

Насколько односимвольных (вставка, удаление, замена) операций две строки отличаются между собой

In [64]:
def levenstein(s1: str, s2: str) -> int:
    n, m = len(s1), len(s2)
    dp = [[0 for _ in range(m+1)] for _ in range(n+1)]
    for i in range(n+1):
        for j in range(m+1):
            if i == 0 and j == 0:
                continue
            elif j == 0 and i > 0:
                dp[i][j] = i
            elif i == 0 and j > 0:
                dp[i][j] = j
            else:
                dp[i][j] = min(
                    dp[i][j-1] + 1,
                    dp[i-1][j] + 1,
                    dp[i-1][j-1] + int(s1[i-1] != s2[j-1])
                )
    return dp[n][m]

s2 = 'gef'
s1 = 'abcd'
levenstein(s2, s1)

4

### 2. Наивный поиск подстроки в строке

In [69]:
def find_substr_naive(s: str, substr: str) -> int:
    if len(substr) > len(s):
        return -1
    n = len(s)
    for i in range(n):
        j = i
        while j < len(substr) and s[j] == substr[j]:
            j += 1
        if j == len(substr):
            return i
    return -1

find_substr_naive('adcdef', 'def')

3

### 3. Z-функция

Z-функция от строки $s$ и позиции $i$ – длина максимального префикса подстроки, начинающейся с позиции $i$ в строке $s$, который одновременно является и префиксом всей строки $s$.

$Z(s, i) = \max k \ | \ s[i \dots i+k] = s[0 \dots k]$

In [70]:
def z_function_naive(s: str) -> list:
    n = len(s)
    zf = [0] * n
    for i in range(1, n):
        while i + zf[i] < n and s[zf[i]] == s[i + zf[i]]:
            zf[i] += 1
    return zf

s = 'abacaba'
z_function_naive(s)

[0, 0, 1, 0, 3, 0, 1]

In [74]:
def z_function(s: str):
    n = len(s)
    zf = [0] * n
    left, right = 0, 0
    for i in range(1, n):
        zf[i] = max(0, min(right - i, zf[i - left]))
        while i + zf[i] < n and s[zf[i]] == s[i + zf[i]]:
            zf[i] += 1
        if i + zf[i] > right:
            left = i
            right = i + zf[i]
        # print(left, right)
        # print(zf)
    return zf

s = 'abacaba'
z_function(s)

[0, 0, 1, 0, 3, 0, 1]

In [81]:
def find_substr(s, substr):
    m, n = len(substr), len(s)
    zf = z_function(f'{substr}#{s}')
    for i in range(m + 1, n + 2):
        if zf[i] == m:
            return i - len(substr) - 1

find_substr('abacaba', 'caba')

3

### 4. Префикс-функция

Префикс-функция от строки $s$ в позиции $i$:

$\pi (s, i) = \max_{k=\overline{1, i}} {k: s[0 \dots k-1 ] = s[i - k + 1 \dots i]}$


In [48]:
def prefix_function_naive(s: str) -> list:
    n = len(s)
    p = [0] * n
    for i in range(n):
        for k in range(i):
            if s[:k] == s[i - k + 1: i + 1]:
                p[i] = k
    return p

s = 'abcabcd'
prefix_function_naive(s)

[0, 0, 0, 1, 2, 3, 0]

In [65]:
def prefix_function(s: str) -> list:
    p = [0] * len(s)
    for i in range(1, len(s)):
        k = p[i - 1]
        while k > 0 and s[i] != s[k]:
            k = p[k - 1]
        if s[i] == s[k]:
            k += 1
        p[i] = k
    return p

In [67]:
def kmp(P: str, T: str) -> list:
    pl, tl = len(P), len(T)
    p = prefix_function_naive(f'{P}#{T}')
    count = 0
    ans = []
    for i in range(tl):
        if p[pl + i + 1] == pl:
            ans.append(i - pl + 1)
            count += 1
    return ans
kmp('caba', 'cabacabacaba')

[0, 4, 8]

Также другие алгоритмы: Нидльмана-Вунша (динамическое программирование), Рабина-Карпа (хэширование), Бойера-Мура, суффиксные массив и дерево